### PDF Document Ingestion

The following example shows using the convert_pdf_to_png_images method to convert a multi-page pdf to an array of images which have been resized to the optimal resolution for PDF documents. This can be used in cases where current Nova Docuemnt Understanding does not support different images that might be present in the document.

#### Setup

We are using a python package - [pdf2image](https://pypi.org/project/pdf2image/) that is an extremely powerful tool to convert pdfs.

This package is dependent on Poppler and needs to be installed first (for other ways to install you can refer to the link)

In [ ]:
%conda install -y -c conda-forge poppler

In [3]:
%pip install pdf2image Pillow 

Looking in indexes: https://pypi.python.org/simple

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Example

In [1]:
import base64
import boto3
import json

from lib.pdf_to_image import convert_pdf_to_png_images

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
)

LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"

# Convert your PDF to images

base64_images = convert_pdf_to_png_images('./immersion_day.pdf')

# Define your system prompt(s).
system_list = [
    {
        "type": "system",
        "content": [{"text": "You will be given images of a document."}],
    }
]

# Create user content block
content = sum([[{"text": f"Image {idx+1}:"}, {"image": {"format": "png", "source": {"bytes": img}}}] for idx, img in enumerate(base64_images)], [])

content.append({
    "text": "Provide a summary of the provided document"
})


# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": content
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

native_request = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

Processed page 1
Processed page 2
[Full Response]
{
  "output": {
    "message": {
      "content": [
        {
          "text": "The document outlines the agenda for the \"Observability Immersion Day\" event, which is part of the AWS Immersion Days v2.0. The event is scheduled for September 5, 2023, and is focused on various AWS observability services. Here is a detailed summary of the agenda:\n\n1. **Introduction to Observability (8:00 AM - 8:10 AM)**\n   - Overview of observability and AWS Observability services.\n\n2. **AWS Observability Options (8:10 AM - 8:20 AM)**\n   - Introduction to the AWS observability suite of services.\n\n3. **CloudWatch Container Insights (8:20 AM - 8:30 AM)**\n   - Use CloudWatch Container Insights to collect, aggregate, and summarize metrics and logs from containerized applications and microservices. Available for Amazon Elastic Container Service (ECS), Amazon Elastic Kubernetes Service (EKS), and Kubernetes on Amazon EC2.\n\n4. **CloudWatch Container